# Hands Signs Digits Classification Using Resnets

In [1]:
# import packages
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

%matplotlib inline
np.random.seed(1)
tf.random.set_seed(2)